In [1]:
import pandas as pd
import numpy as np
import faiss 

In [2]:
import faiss
print(faiss.__version__)

1.11.0


### EDA (Faults)

In [4]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory=False)

In [5]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [6]:
faults.shape

(1187335, 20)

In [7]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187335 non-null  int64  
 1   ESS_Id                 1187335 non-null  int64  
 2   EventTimeStamp         1187335 non-null  object 
 3   eventDescription       1126490 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891285 non-null   object 
 6   ecuSerialNumber        844318 non-null   object 
 7   ecuModel               1122577 non-null  object 
 8   ecuMake                1122577 non-null  object 
 9   ecuSource              1187335 non-null  int64  
 10  spn                    1187335 non-null  int64  
 11  fmi                    1187335 non-null  int64  
 12  active                 1187335 non-null  bool   
 13  activeTransitionCount  1187335 non-null  int64  
 14  faultValue        

In [8]:
faults.value_counts(normalize= True)

Series([], Name: proportion, dtype: float64)

In [9]:
faults.isnull().all() # checking to see which columns are completely NaN values
                      # And there are two: actionDescription, faultValue  - i'll drop these

RecordID                 False
ESS_Id                   False
EventTimeStamp           False
eventDescription         False
actionDescription         True
ecuSoftwareVersion       False
ecuSerialNumber          False
ecuModel                 False
ecuMake                  False
ecuSource                False
spn                      False
fmi                      False
active                   False
activeTransitionCount    False
faultValue                True
EquipmentID              False
MCTNumber                False
Latitude                 False
Longitude                False
LocationTimeStamp        False
dtype: bool

In [10]:
faults.isnull().all(axis=1).value_counts() # checking to see if there are any rows that are completely NaN values
                                            # And there aren't any

False    1187335
Name: count, dtype: int64

In [11]:
faults.loc[0]

RecordID                                                       1
ESS_Id                                                    990349
EventTimeStamp                           2015-02-21 10:47:13.000
eventDescription         Low (Severity Low) Engine Coolant Level
actionDescription                                            NaN
ecuSoftwareVersion                                       unknown
ecuSerialNumber                                          unknown
ecuModel                                                 unknown
ecuMake                                                  unknown
ecuSource                                                      0
spn                                                          111
fmi                                                           17
active                                                      True
activeTransitionCount                                          2
faultValue                                                   NaN
EquipmentID              

Looking at the first record, here is a breakdown of the important values.

* ESS_Id, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, faultValue, and MCTNumber are unlikely to provide any predictive value.
* We can see the time of the event in the **EventTimeStamp** column. Note that this may be different from the **LocationTimeStamp** value, which indicates when the Latitude/Longitude values were recorded.
* The **spn** and **fmi** columns together indicate the type of fault, and there may be a description of that fault in the **eventDescription** column, although this column is sometimes missing.
* Faults are recorded when the light goes on and when it goes off, which is indicated by the **active** column, with True indicating the light turning on and False indicating turning off. The number of times the code has been set or unset is in the **faultValue** column, although this value can be unreliable. 
* Each truck has an identifier, the **EquipmentID** value.
* Each record can be linked to the on-board diagnostics data through the **RecordID** column.

In [13]:
print(faults.duplicated().value_counts(normalize= True))
print(faults[faults.duplicated()]) 
# I am trying to identify duplicated rows and there are none, so I don't need to drop any. 

False    1.0
Name: proportion, dtype: float64
Empty DataFrame
Columns: [RecordID, ESS_Id, EventTimeStamp, eventDescription, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, spn, fmi, active, activeTransitionCount, faultValue, EquipmentID, MCTNumber, Latitude, Longitude, LocationTimeStamp]
Index: []


In [14]:
# print(faults['ESS_Id'].isnull().value_counts(normalize = True))
# print(faults['actionDescription'].isnull().value_counts(normalize = True))
# print(faults['ecuSoftwareVersion'].isnull().value_counts(normalize = True))
# print(faults['ecuSerialNumber'].isnull().value_counts(normalize = True))
# print(faults['ecuModel'].isnull().value_counts(normalize = True))
# print(faults['ecuMake'].isnull().value_counts(normalize = True))
# print(faults['ecuSource'].isnull().value_counts(normalize = True))
# print(faults['faultValue'].isnull().value_counts(normalize = True))
# print(faults['MCTNumber'].isnull().value_counts(normalize = True))

### Data Prepping (Faults)

In [16]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')
faults_prepped.shape
# dropped columns that contained only NaN values # dropped rows that contained only NaN values

(1187335, 18)

### EDA (Diagnostics)

In [18]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

In [19]:
diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [20]:
diagnostics.shape

(12821626, 4)

In [21]:
diagnostics['Name'].value_counts().count()

24

In [22]:
faults['RecordID'].value_counts(normalize=True)

RecordID
1          8.422223e-07
812048     8.422223e-07
812100     8.422223e-07
812099     8.422223e-07
812098     8.422223e-07
               ...     
403279     8.422223e-07
403278     8.422223e-07
403277     8.422223e-07
403276     8.422223e-07
1248458    8.422223e-07
Name: proportion, Length: 1187335, dtype: float64

In [23]:
diagnostics['FaultId'].value_counts()

FaultId
206596     23
34901      23
163938     23
163939     23
163946     23
           ..
762004      1
328737      1
328735      1
762008      1
1248458     1
Name: count, Length: 1187335, dtype: int64

In [24]:
diagnostics.info()
# there are no NaN columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


### Data Prepping (Diagnostics)

In [26]:
print(diagnostics.duplicated().value_counts(normalize= True))
print(diagnostics[diagnostics.duplicated()]) 

False    1.0
Name: proportion, dtype: float64
Empty DataFrame
Columns: [Id, Name, Value, FaultId]
Index: []


In [27]:
diagnostics_prepped = diagnostics.dropna(axis=1, how='all').dropna(axis=0, how='all')
diagnostics_prepped.shape

(12821626, 4)

### Faults & Diagnostic Data Prepping

In [29]:
diagnostics_pivoted = diagnostics.pivot(index = 'FaultId', columns='Name', values='Value').reset_index()
diagnostics_pivoted

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [30]:
faults_diagnostics = pd.merge(faults_prepped, diagnostics_pivoted, left_on='RecordID', right_on='FaultId', how='inner')
faults_diagnostics.shape

(1187335, 43)

In [31]:
print(f"There are this many duplicated values:", faults_diagnostics.duplicated().value_counts()) # there are no duplicated rows. tbe.
faults_diagnostics.isnull().all() # there are no completely NaN columns.

There are this many duplicated values: False    1187335
Name: count, dtype: int64


RecordID                     False
ESS_Id                       False
EventTimeStamp               False
eventDescription             False
ecuSoftwareVersion           False
ecuSerialNumber              False
ecuModel                     False
ecuMake                      False
ecuSource                    False
spn                          False
fmi                          False
active                       False
activeTransitionCount        False
EquipmentID                  False
MCTNumber                    False
Latitude                     False
Longitude                    False
LocationTimeStamp            False
FaultId                      False
AcceleratorPedal             False
BarometricPressure           False
CruiseControlActive          False
CruiseControlSetSpeed        False
DistanceLtd                  False
EngineCoolantTemperature     False
EngineLoad                   False
EngineOilPressure            False
EngineOilTemperature         False
EngineRpm           

In [32]:
faults_diagnostics.isnull().all(axis=1).value_counts() # there are no completely empty rows. 

False    1187335
Name: count, dtype: int64

In [33]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   RecordID                   1187335 non-null  int64  
 1   ESS_Id                     1187335 non-null  int64  
 2   EventTimeStamp             1187335 non-null  object 
 3   eventDescription           1126490 non-null  object 
 4   ecuSoftwareVersion         891285 non-null   object 
 5   ecuSerialNumber            844318 non-null   object 
 6   ecuModel                   1122577 non-null  object 
 7   ecuMake                    1122577 non-null  object 
 8   ecuSource                  1187335 non-null  int64  
 9   spn                        1187335 non-null  int64  
 10  fmi                        1187335 non-null  int64  
 11  active                     1187335 non-null  bool   
 12  activeTransitionCount      1187335 non-null  int64  
 13  EquipmentID 

In [34]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3958.7  # roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2]) # applies function to an iterable
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle


threshold = 0.5  # this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_diagnostics['Latitude'], faults_diagnostics['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).

# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_diagnostics = faults_diagnostics[outside_vicinity]

faults_diagnostics

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [35]:
print(f"the amount of rows removed because they're within 0.5 miles are:", 1187335 - 1057049)
distances.dtype

the amount of rows removed because they're within 0.5 miles are: 130286


dtype('float64')

In [36]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057049 entries, 0 to 1187334
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   RecordID                   1057049 non-null  int64  
 1   ESS_Id                     1057049 non-null  int64  
 2   EventTimeStamp             1057049 non-null  object 
 3   eventDescription           1006152 non-null  object 
 4   ecuSoftwareVersion         792713 non-null   object 
 5   ecuSerialNumber            751116 non-null   object 
 6   ecuModel                   1001026 non-null  object 
 7   ecuMake                    1001026 non-null  object 
 8   ecuSource                  1057049 non-null  int64  
 9   spn                        1057049 non-null  int64  
 10  fmi                        1057049 non-null  int64  
 11  active                     1057049 non-null  bool   
 12  activeTransitionCount      1057049 non-null  int64  
 13  EquipmentID      

In [37]:
columns_to_change_to_float = ['AcceleratorPedal', 
                     'BarometricPressure',
                     'CruiseControlSetSpeed',
                     'DistanceLtd',
                     'EngineCoolantTemperature',
                     'EngineLoad',
                     'EngineOilPressure',
                     'EngineOilTemperature',
                     'EngineRpm',
                     'EngineTimeLtd',
                     'FuelLevel',
                     'FuelLtd',
                     'FuelRate',
                     'FuelTemperature',
                     'IntakeManifoldTemperature',
                     'Speed',
                     'SwitchedBatteryVoltage',
                     'TurboBoostPressure',
                     'Throttle'
                    ]

In [38]:
for column in columns_to_change_to_float:
    faults_diagnostics[column] = faults_diagnostics[column].str.replace(',', '.').astype(float)

In [39]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057049 entries, 0 to 1187334
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   RecordID                   1057049 non-null  int64  
 1   ESS_Id                     1057049 non-null  int64  
 2   EventTimeStamp             1057049 non-null  object 
 3   eventDescription           1006152 non-null  object 
 4   ecuSoftwareVersion         792713 non-null   object 
 5   ecuSerialNumber            751116 non-null   object 
 6   ecuModel                   1001026 non-null  object 
 7   ecuMake                    1001026 non-null  object 
 8   ecuSource                  1057049 non-null  int64  
 9   spn                        1057049 non-null  int64  
 10  fmi                        1057049 non-null  int64  
 11  active                     1057049 non-null  bool   
 12  activeTransitionCount      1057049 non-null  int64  
 13  EquipmentID      

In [40]:
faults_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_diagnostics['EventTimeStamp'])
faults_diagnostics['LocationTimeStamp'] = pd.to_datetime(faults_diagnostics['LocationTimeStamp'])

In [41]:
object_columns_to_change_to_bool = [
    'CruiseControlActive', 
    'IgnStatus', 
    'ParkingBrake',
    'active'
]

In [42]:
for column in object_columns_to_change_to_bool:
    faults_diagnostics[column] = faults_diagnostics[column].astype(bool)

In [43]:
object_columns_to_change_to_category = [        
    'eventDescription',  
    'ecuSoftwareVersion',          
    'ecuSerialNumber',         
    'ecuModel',           
    'ecuMake',                  
    'EquipmentID',                
    'LocationTimeStamp',  
    'LampStatus',                         
    'ServiceDistance'
]             

In [44]:
for column in object_columns_to_change_to_category:
    faults_diagnostics[column] = faults_diagnostics[column].astype('category')

In [45]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057049 entries, 0 to 1187334
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1057049 non-null  int64         
 1   ESS_Id                     1057049 non-null  int64         
 2   EventTimeStamp             1057049 non-null  datetime64[ns]
 3   eventDescription           1006152 non-null  category      
 4   ecuSoftwareVersion         792713 non-null   category      
 5   ecuSerialNumber            751116 non-null   category      
 6   ecuModel                   1001026 non-null  category      
 7   ecuMake                    1001026 non-null  category      
 8   ecuSource                  1057049 non-null  int64         
 9   spn                        1057049 non-null  int64         
 10  fmi                        1057049 non-null  int64         
 11  active                     1057049 non-nul

In [108]:
faults_diagnostics.head()
faults_diagnostics['next_derate_timestamp'] = faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,next_derate_timestamp,time_until_derate,target
1154193,1211417,108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,...,1279,True,NaN,0.0,NaN,100.0,0.58,NaT,NaT,False
1154194,1211418,108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,1279,True,NaN,0.0,NaN,100.0,0.58,NaT,NaT,False
1154196,1211420,108604488,2000-03-18 19:20:47,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,...,255,True,NaN,NaN,NaN,NaN,NaN,NaT,NaT,False
1154195,1211419,108604487,2000-03-18 19:20:47,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,255,True,NaN,NaN,NaN,NaN,NaN,NaT,NaT,False
1154198,1211422,108608408,2000-03-19 02:59:58,Not Reporting Data Wheel Sensor ABS Axle 2 Right,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,5W26153559,EC80ESP,BNDWS,11,792,...,1279,True,NaN,NaN,NaN,NaN,NaN,NaT,NaT,False


In [46]:
faults_diagnostics['next_derate_timestamp'] = faults_diagnostics.loc[faults_diagnostics['spn'] == 5246]['EventTimeStamp']
print(F"The amount of SPN date values are {faults_diagnostics['next_derate_timestamp'].notna().sum()}")
print(f"The amount of NaNs (dates when spn didn't occur) {faults_diagnostics['next_derate_timestamp'].isna().sum()}")
faults_diagnostics = faults_diagnostics.sort_values('EventTimeStamp')
print(F"bfill leaves us with {faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill().isna().value_counts().iloc[0]} nans and {faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill().isna().value_counts().iloc[1]} date values in next_derate_timestamp column")
faults_diagnostics['next_derate_timestamp'] = faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill()
print(F"bfill + ffill leaves us with {faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill().ffill().isna().value_counts().iloc[1]} nans and {faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill().ffill().isna().value_counts().iloc[0]} date values in next_derate_timestamp column")
faults_diagnostics['next_derate_timestamp'] = faults_diagnostics.groupby('EquipmentID', observed=True)['next_derate_timestamp'].bfill().ffill()
print("We're going with bfill+ffill")

The amount of SPN date values are 936
The amount of NaNs (dates when spn didn't occur) 1056113
bfill leaves us with 807998 nans and 249051 date values in next_derate_timestamp column
bfill + ffill leaves us with 16 nans and 1057033 date values in next_derate_timestamp column
We're going with bfill+ffill


In [47]:
faults_diagnostics['time_until_derate'] = faults_diagnostics['next_derate_timestamp'] - faults_diagnostics['EventTimeStamp']

In [118]:
faults_diagnostics.loc[faults_diagnostics['time_until_derate'] < pd.Timedelta(0)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,next_derate_timestamp,time_until_derate,target
1182803,1243927,121506555,2011-01-01 00:03:25,Low (Severity Medium) Engine Coolant Level,04358814*06008736*100515184137*09400147*G1*BDR*,79897312,6X1u13D1500000000,CMMNS,0,111,...,18431,True,NaN,1.330123,NaN,100.0,0.87,2011-01-01 00:03:23,-1 days +23:59:58,False
941188,970714,40551214,2011-01-01 00:03:26,Incorrect Data Ambient Air Temperature,NaN,NaN,CECU3B-NAMUX4,PACCR,49,171,...,255,True,NaN,NaN,NaN,NaN,NaN,2011-01-01 00:03:23,-1 days +23:59:57,False
669953,687181,13908742,2011-01-01 00:03:29,NaN,04358814*06088094*061516161145*09401661*G1*BDR*,79909430,6X1u13D1500000000,CMMNS,0,5394,...,17407,True,NaN,0.000000,NaN,100.0,0.29,2011-01-01 00:03:23,-1 days +23:59:54,False
3013,3014,1026825,2015-02-23 15:54:48,Incorrect Data Warning Light Bulb,unknown,unknown,unknown,unknown,11,811,...,1279,True,NaN,NaN,NaN,NaN,NaN,2015-02-23 15:54:22,-1 days +23:59:34,False
5810,5811,1071908,2015-02-25 14:47:20,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,...,17407,True,NaN,NaN,NaN,NaN,NaN,2015-02-25 14:47:00,-1 days +23:59:40,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,...,1023,True,NaN,NaN,NaN,NaN,NaN,2020-02-17 17:08:39,-18 days +03:08:13,False
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,18431,True,NaN,65.010960,NaN,73.2,7.83,2020-02-17 17:08:39,-18 days +03:04:16,False
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,17407,True,NaN,66.574100,NaN,100.0,6.96,2020-02-17 17:08:39,-18 days +02:55:01,False
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,...,1023,True,NaN,11.844890,14.1,100.0,1.74,2020-02-17 17:08:39,-18 days +02:54:26,False


In [48]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057049 entries, 1154193 to 1187334
Data columns (total 45 columns):
 #   Column                     Non-Null Count    Dtype          
---  ------                     --------------    -----          
 0   RecordID                   1057049 non-null  int64          
 1   ESS_Id                     1057049 non-null  int64          
 2   EventTimeStamp             1057049 non-null  datetime64[ns] 
 3   eventDescription           1006152 non-null  category       
 4   ecuSoftwareVersion         792713 non-null   category       
 5   ecuSerialNumber            751116 non-null   category       
 6   ecuModel                   1001026 non-null  category       
 7   ecuMake                    1001026 non-null  category       
 8   ecuSource                  1057049 non-null  int64          
 9   spn                        1057049 non-null  int64          
 10  fmi                        1057049 non-null  int64          
 11  active                 

In [49]:
faults_diagnostics['target'] = (faults_diagnostics['time_until_derate'] > '2 hour') & (faults_diagnostics['time_until_derate'] < '24 hour')

In [50]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057049 entries, 1154193 to 1187334
Data columns (total 46 columns):
 #   Column                     Non-Null Count    Dtype          
---  ------                     --------------    -----          
 0   RecordID                   1057049 non-null  int64          
 1   ESS_Id                     1057049 non-null  int64          
 2   EventTimeStamp             1057049 non-null  datetime64[ns] 
 3   eventDescription           1006152 non-null  category       
 4   ecuSoftwareVersion         792713 non-null   category       
 5   ecuSerialNumber            751116 non-null   category       
 6   ecuModel                   1001026 non-null  category       
 7   ecuMake                    1001026 non-null  category       
 8   ecuSource                  1057049 non-null  int64          
 9   spn                        1057049 non-null  int64          
 10  fmi                        1057049 non-null  int64          
 11  active                 

In [51]:
print(faults_diagnostics['Throttle'].isna().mean())
print("isna() returns a boolean series of 1 True 0 False. When mean() is used on a boolean series it returns a proportion.")

0.6448244121133457
isna() returns a boolean series of 1 True 0 False. When mean() is used on a boolean series it returns a proportion.


In [52]:
nan_threshold = 0.7
faults_diagnostics_threshed70 = faults_diagnostics.loc[:, faults_diagnostics.isna().mean() < nan_threshold]
dropped_columns = set(faults_diagnostics.columns) - set(faults_diagnostics_threshed70.columns)
print(F"The amount of columns before thresholding {faults_diagnostics.shape[1]} and the amount after thresholding for 70%: {faults_diagnostics_threshed70.shape[1]}")
print(F"the dropped columns are: {dropped_columns}")
print(F"the shape: {faults_diagnostics_threshed70.shape}")
print(F"The count of NaN values: {faults_diagnostics_threshed70.isna().sum()}")
faults_diagnostics_threshed70.to_csv('../data/data_threshed70.csv')

The amount of columns before thresholding 46 and the amount after thresholding for 70%: 43
the dropped columns are: {'FuelTemperature', 'ServiceDistance', 'SwitchedBatteryVoltage'}
the shape: (1057049, 43)
The count of NaN values: RecordID                          0
ESS_Id                            0
EventTimeStamp                    0
eventDescription              50897
ecuSoftwareVersion           264336
ecuSerialNumber              305933
ecuModel                      56023
ecuMake                       56023
ecuSource                         0
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
EquipmentID                       0
MCTNumber                         0
Latitude                          0
Longitude                         0
LocationTimeStamp                 0
FaultId                           0
AcceleratorPedal             576389
BarometricPressure           526147
CruiseControl

In [106]:
faults_diagnostics_threshed70.dropna()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,next_derate_timestamp,time_until_derate,target


In [53]:
nan_threshold = 0.5
faults_diagnostics_threshed50 = faults_diagnostics.loc[:, faults_diagnostics.isna().mean() < nan_threshold]
dropped_columns = set(faults_diagnostics.columns) - set(faults_diagnostics_threshed50.columns)
print(F"The amount of columns before thresholding {faults_diagnostics.shape[1]} and the amount after thresholding for 50%: {faults_diagnostics_threshed50.shape[1]}")
print(F"the dropped columns are: {dropped_columns}")
print(F"the shape: {faults_diagnostics_threshed50.shape}")
print(F"The count of NaN values: {faults_diagnostics_threshed50.isna().sum()}")
faults_diagnostics_threshed50.to_csv('../data/data_threshed50.csv')

The amount of columns before thresholding 46 and the amount after thresholding for 50%: 38
the dropped columns are: {'CruiseControlSetSpeed', 'AcceleratorPedal', 'SwitchedBatteryVoltage', 'EngineTimeLtd', 'FuelTemperature', 'FuelLevel', 'Throttle', 'ServiceDistance'}
the shape: (1057049, 38)
The count of NaN values: RecordID                          0
ESS_Id                            0
EventTimeStamp                    0
eventDescription              50897
ecuSoftwareVersion           264336
ecuSerialNumber              305933
ecuModel                      56023
ecuMake                       56023
ecuSource                         0
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
EquipmentID                       0
MCTNumber                         0
Latitude                          0
Longitude                         0
LocationTimeStamp                 0
FaultId                           

In [54]:
nan_threshold = 0.3
faults_diagnostics_threshed30 = faults_diagnostics.loc[:, faults_diagnostics.isna().mean() < nan_threshold]
dropped_columns = set(faults_diagnostics.columns) - set(faults_diagnostics_threshed30.columns)
print(F"The amount of columns before thresholding {faults_diagnostics.shape[1]} and the amount after thresholding for 30% {faults_diagnostics_threshed30.shape[1]}")
print(F"the dropped columns are: {dropped_columns}")
print(F"the shape: {faults_diagnostics_threshed30.shape}")
print(F"The count of NaN values: {faults_diagnostics_threshed30.isna().sum()}")
faults_diagnostics_threshed30.to_csv('../data/data_threshed30.csv')

The amount of columns before thresholding 46 and the amount after thresholding for 30% 26
the dropped columns are: {'FuelTemperature', 'EngineLoad', 'FuelLtd', 'ServiceDistance', 'EngineRpm', 'Speed', 'IntakeManifoldTemperature', 'FuelRate', 'SwitchedBatteryVoltage', 'EngineTimeLtd', 'BarometricPressure', 'FuelLevel', 'EngineOilTemperature', 'DistanceLtd', 'TurboBoostPressure', 'CruiseControlSetSpeed', 'AcceleratorPedal', 'EngineOilPressure', 'Throttle', 'EngineCoolantTemperature'}
the shape: (1057049, 26)
The count of NaN values: RecordID                      0
ESS_Id                        0
EventTimeStamp                0
eventDescription          50897
ecuSoftwareVersion       264336
ecuSerialNumber          305933
ecuModel                  56023
ecuMake                   56023
ecuSource                     0
spn                           0
fmi                           0
active                        0
activeTransitionCount         0
EquipmentID                   0
MCTNumber      